<a href="https://colab.research.google.com/github/kingy0ujin/2023-A-DA/blob/main/inclass/12w_0516_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFA 선수 정보를 통한 포지션 예측 -> 앙상블 기법으로

In [76]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [77]:
df = pd.read_csv("./fifa_data.csv")
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [78]:
df1 = pd.DataFrame({'Name' : df.Name, 'Club' : df.Club, 'Position' : df.Position})
df2 = df.iloc[:, 54:88].astype(float) # 스텟데이터

td = pd.concat([df1,df2], axis=1)
td

# 코드 대신 엑셀에서 수정해도 됨
# 선수 정보 데이터 위주로 보

,Name,Club,Position,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,L. Messi,FC Barcelona,RF,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,Cristiano Ronaldo,Juventus,ST,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,Neymar Jr,Paris Saint-Germain,LW,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,De Gea,Manchester United,GK,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,K. De Bruyne,Manchester City,RCM,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,J. Lundstram,Crewe Alexandra,CM,34.0,38.0,40.0,49.0,25.0,42.0,30.0,...,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0
18203,N. Christoffersson,Trelleborgs FF,ST,23.0,52.0,52.0,43.0,36.0,39.0,32.0,...,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0
18204,B. Worman,Cambridge United,ST,25.0,40.0,46.0,38.0,38.0,45.0,38.0,...,55.0,41.0,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0
18205,D. Walker-Rice,Tranmere Rovers,RW,44.0,50.0,39.0,42.0,40.0,51.0,34.0,...,50.0,46.0,20.0,25.0,27.0,14.0,6.0,14.0,8.0,9.0


In [79]:
tmp = td.columns

del td[tmp[9]]
del td[tmp[10]]
del td[tmp[11]]
del td[tmp[15]]
del td[tmp[16]]
del td[tmp[17]]
del td[tmp[18]]
del td[tmp[19]]
del td[tmp[20]]
del td[tmp[21]]
del td[tmp[22]]
del td[tmp[23]]
del td[tmp[28]]

td.columns

Index(['Name', 'Club', 'Position', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes'],
      dtype='object')

In [80]:
# 현재 가지고 있는 데이터에서 각 포지션의 갯수를 확인한다
td.Position.value_counts()

# LF 포지션의 개수가 15개로 작음 -> 더 많은 데이터를 fifa 측에 요청하거나 아예 삭제해버리기

Position
ST     2152
GK     2025
CB     1778
CM     1394
LB     1322
RB     1291
RM     1124
LM     1095
CAM     958
CDM     948
RCB     662
LCB     648
LCM     395
RCM     391
LW      381
RW      370
RDM     248
LDM     243
LS      207
RS      203
RWB      87
LWB      78
CF       74
LAM      21
RAM      21
RF       16
LF       15
Name: count, dtype: int64

In [81]:
td.loc[td['Position'] == 'LF', ['Position']] = 'ST'
td.loc[td['Position'] == 'RF', ['Position']] = 'ST'
td.loc[td['Position'] == 'CF', ['Position']] = 'ST'
td.loc[td['Position'] == 'LS', ['Position']] = 'ST'
td.loc[td['Position'] == 'RS', ['Position']] = 'ST'
td.loc[td['Position'] == 'LAM', ['Position']] = 'CAM'
td.loc[td['Position'] == 'RAM', ['Position']] = 'CAM'
td.loc[td['Position'] == 'LCM', ['Position']] = 'CM'
td.loc[td['Position'] == 'RCM', ['Position']] = 'CM'
td.loc[td['Position'] == 'RDM', ['Position']] = 'CDM'
td.loc[td['Position'] == 'LDM', ['Position']] = 'CDM'
td.loc[td['Position'] == 'LW', ['Position']] = 'WF'
td.loc[td['Position'] == 'RW', ['Position']] = 'WF'
td.loc[td['Position'] == 'LB', ['Position']] = 'WB'
td.loc[td['Position'] == 'RB', ['Position']] = 'WB'
td.loc[td['Position'] == 'LWB', ['Position']] = 'WB'
td.loc[td['Position'] == 'RWB', ['Position']] = 'WB'
td.loc[td['Position'] == 'LM', ['Position']] = 'WM'
td.loc[td['Position'] == 'RM', ['Position']] = 'WM'
td.loc[td['Position'] == 'LCB', ['Position']] = 'CB'
td.loc[td['Position'] == 'RCB', ['Position']] = 'CB'

In [82]:
td.Position.value_counts()

Position
CB     3088
WB     2778
ST     2667
WM     2219
CM     2180
GK     2025
CDM    1439
CAM    1000
WF      751
Name: count, dtype: int64

In [83]:
td = td.dropna() # null 값 제거

In [84]:
# sklearn의 train_test_split 을 사용하면 라인 한줄로 손쉽게 데이터를 나눌 수 있다
from sklearn.model_selection import train_test_split

# 다듬어진 데이터에서 20%를 테스트 데이터로 분류합니다
train, test = train_test_split(td, test_size=0.2)

In [85]:
# 학습 데이터의 갯수를 확인, 14565개의 데이터가 있습니다.
train.shape[0]

14334

In [86]:
# 테스트 데이터의 갯수를 확인, 3642개의 데이터가 있습니다
test.shape[0]

3584

In [87]:
with open('fifa_train.pkl', 'wb') as train_data:
  pickle.dump(train, train_data)

with open('fifa_test.pkl', 'wb') as test_data:
  pickle.dump(test, test_data)

# 파일로 데이터 저장 해 놓기 -> 여러 팀과 협업할 때 사용하기 위해서

In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

import numpy as np

In [89]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

random_forest_clf = RandomForestClassifier(random_state=42)
extra_trees_clf = ExtraTreesClassifier(random_state=42)
svm_clf = SVC(kernel="rbf", degree=3, coef0 = 0.0, C=1.0, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
  print("훈련 모델:", estimator)
  estimator.fit(X_train, y_train)

[estimator.score(X_test, y_test) for estimator in estimators]

훈련 모델: RandomForestClassifier(random_state=42)


<ipython-input-89-1cc786929aa1>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train)


훈련 모델: ExtraTreesClassifier(random_state=42)


<ipython-input-89-1cc786929aa1>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train)


훈련 모델: SVC(random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 모델: MLPClassifier(random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.7251674107142857,
 0.7195870535714286,
 0.7318638392857143,
 0.7117745535714286]

In [90]:
# 앙상블 학습
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]
voting_clf = VotingClassifier(named_estimators)

voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0.7265625

In [91]:
# @title 기본 제목 텍스트
# 테스트에 사용될 특징을 지정합니다
X_test = test.iloc[:,3:24]

# 특징으로 예측할 값(포지션)을 지정
y_test = test[['Position']]

# 최적의 파라미터를 완성된 SVM에 테스트 데이터를 주입하여, 실제값과 예측값을 얻습니다.
y_true, y_pred = y_test, clf.predict(X_test)

print('accuracy : '+str(accuracy_score(y_true, y_pred)))

accuracy : 0.7056361607142857


In [92]:
# 실제 값(ground truth)과 예측값(prediction)이 어느 정도 일치하는지 비교
comparison = pd.DataFrame({'pedictation' : y_pred, 'ground_truth' : y_true.values.ravel()})
comparison


,pedictation,ground_truth
0,CM,CB
1,WM,CDM
2,WM,WF
3,ST,ST
4,WB,WB
...,...,...
3579,WB,WF
3580,WB,WB
3581,CM,CDM
3582,WM,WF


In [93]:
comparison.to_csv('result.csv')